***CREATE RECORD AUDIT DATA***

In [0]:
config = spark.read.option("multiline", "true").json("dbfs:/configs/config.json")
env = config.first()["env"].strip().lower()
lz_key = config.first()["lz_key"].strip().lower()

keyvault_name = f"ingest{lz_key}-meta002-{env}"

# Access the Service Principle secrets from keyvaults
client_secret = dbutils.secrets.get(scope=keyvault_name, key='SERVICE-PRINCIPLE-CLIENT-SECRET')
tenant_id = dbutils.secrets.get(scope=keyvault_name, key='SERVICE-PRINCIPLE-TENANT-ID')
client_id = dbutils.secrets.get(scope=keyvault_name, key='SERVICE-PRINCIPLE-CLIENT-ID')

storage_accounts = ["curated"]

In [0]:
for storage_account in storage_accounts:
    spark.conf.set(
        f'fs.azure.account.auth.type.ingest{lz_key}{storage_account}{env}.dfs.core.windows.net', 'OAuth')
    spark.conf.set(
        f'fs.azure.account.oauth.provider.type.ingest{lz_key}{storage_account}{env}.dfs.core.windows.net', 'org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider')
    spark.conf.set(
        f'fs.azure.account.oauth2.client.id.ingest{lz_key}{storage_account}{env}.dfs.core.windows.net', client_id)
    spark.conf.set(
        f'fs.azure.account.oauth2.client.secret.ingest{lz_key}{storage_account}{env}.dfs.core.windows.net', client_secret)
    spark.conf.set(
        f'fs.azure.account.oauth2.client.endpoint.ingest{lz_key}{storage_account}{env}.dfs.core.windows.net', f'https://login.microsoftonline.com/{tenant_id}/oauth2/token')

In [0]:
ARM_segment = "FPADEV" if env == "sbox" else "FPA"
ARIA_segment = "APLFPA"

In [0]:
curated_storage_account = f"ingest{lz_key}curated{env}"
silver_container = "silver"

In [0]:
df_create_record_data = spark.read.format("delta").load(f"abfss://{silver_container}@{curated_storage_account}.dfs.core.windows.net/ARIADM/ARM/response/{ARIA_segment}/create_record/")
df_create_record_data.createOrReplaceTempView("fpa_create_record")

In [0]:
%sql

SELECT  DATE_FORMAT(Process_time, 'ddMMMyyyy') as date,
        operation,
        COUNT(*) as total_expected_count,
        COUNT(CASE WHEN status = 1 THEN 1 END) as count_successful_response,
        COUNT(CASE WHEN status = 0 THEN 1 END) as count_unsuccessful_response,
        COUNT(CASE WHEN status IS NULL THEN 1 END) as count_status_null

FROM fpa_create_record

GROUP BY DATE_FORMAT(Process_time, 'ddMMMyyyy'), operation

Databricks visualization. Run in Databricks to view.

***INPUT UPLOAD AUDIT DATA***

In [0]:
df_input_upload_data = spark.read.format("delta").load(f"abfss://{silver_container}@{curated_storage_account}.dfs.core.windows.net/ARIADM/ARM/response/{ARIA_segment}/input_upload/")
df_input_upload_data.createOrReplaceTempView("fpa_input_upload")

In [0]:
%sql

SELECT  DATE_FORMAT(timestamp, 'ddMMMyyyy') as date,
        operation,
        COUNT(*) as total_expected_count,
        COUNT(CASE WHEN status = 1 THEN 1 END) as count_successful_response,
        COUNT(CASE WHEN status = 0 THEN 1 END) as count_unsuccessful_response,
        COUNT(CASE WHEN status IS NULL THEN 1 END) as count_status_null

FROM fpa_input_upload

GROUP BY DATE_FORMAT(timestamp, 'ddMMMyyyy'), operation

Databricks visualization. Run in Databricks to view.

***UPLOAD FILE***

In [0]:
df_upload_file_data = spark.read.format("delta").load(f"abfss://{silver_container}@{curated_storage_account}.dfs.core.windows.net/ARIADM/ARM/response/{ARIA_segment}/upload_file/")
df_upload_file_data.createOrReplaceTempView("fpa_upload_file")

In [0]:
%sql

SELECT  DATE_FORMAT(process_time, 'ddMMMyyyy') as date,
        operation,
        COUNT(*) as total_expected_count,
        SUM(CASE WHEN status = 1 THEN 1 END) as count_successful_response,
        SUM(CASE WHEN status = 0 THEN 1 END) as count_unsuccessful_response,
        SUM(CASE WHEN status IS NULL THEN 1 END) as count_status_null

FROM fpa_upload_file

GROUP BY DATE_FORMAT(process_time, 'ddMMMyyyy'), operation

Databricks visualization. Run in Databricks to view.

***AMALGAMATED AUDIT DATA***

In [0]:
df_amalgamated_response_data = spark.read.format("delta").load(f"abfss://{silver_container}@{curated_storage_account}.dfs.core.windows.net/ARIADM/ARM/response/{ARIA_segment}/amalgamated_responses/")
df_amalgamated_response_data.createOrReplaceTempView("fpa_amalgamated_response_data")

In [0]:
%sql 

SELECT 
        DATE_FORMAT(process_time, 'ddMMMyyyy') as date,
        operation,
        COUNT(CASE WHEN operation = 'create_record' THEN 1 END) AS count_of_create_record,
        COUNT(CASE WHEN operation = 'upload_new_file' THEN 1 END) AS count_of_upload_new_file,
        COUNT(CASE WHEN operation = 'input_upload' THEN 1 END) AS count_of_input_upload,
        COUNT(CASE WHEN status = 1 THEN 1 END) AS count_of_active,
        COUNT(*) AS total_expected_count

 FROM fpa_amalgamated_response_data

 GROUP BY DATE_FORMAT(process_time, 'ddMMMyyyy'), operation

In [0]:
%sql


SELECT * FROM fpa_amalgamated_response_data